In [10]:
import os 
import re 
import cantera as ct

directory ='../'
species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}


flame_speed_calc = '''

import cantera as ct
import numpy as np
import pandas as pd

print("Running Cantera Version: " + str(ct.__version__))

To = 298
Po = 1e5 # ct.one_atm

gas = ct.Solution('./chemkin/chem_final.cti')

'''


def make_vol_frac_list(vol_frac_lower_bound, vol_frac_upper_bound):

    block = \
   '''
i = ***vol_frac_lower_bound***
mole_frac_list = []

while i < ***vol_frac_upper_bound***: 
   mole_frac_list.append(i)
   i += 0.0025 


'''.replace('***vol_frac_lower_bound***',str(vol_frac_lower_bound)).replace('***vol_frac_upper_bound***',str(vol_frac_upper_bound))
    
    return block

def make_for_loop(label1, label2, label3):

   block = \
   '''
results = {}

for x in mole_frac_list: 
    norm_ox = (1-x)*.21
    mole_frac_dict = {'***label1***': (x/norm_ox), '***label2***': (x/norm_ox), '***label3***':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox } 
    gas.TPX = To, Po, mole_frac_dict
    width = 0.08
    flame = ct.FreeFlame(gas, width=width)
    flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
    flame.max_time_step_count = 900
    loglevel = 1 
    flame.solve(loglevel=loglevel, auto=True)
    Su = flame.u[0]
    results[x] = Su
    sltn = flame.to_solution_array()
    pd = sltn.to_pandas()
    pd.to_csv(f'data_6d/test_{x}.csv')

vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)

'''.replace('***label1***',label1).replace('***label2***',label2).replace('***label3***',label3) 
   return block

def make_bash_script(label1,label2):
    bash_script = """#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time=7-00:00:00
#SBATCH --job-name=fc6d_label1_label2
#SBATCH --error=fc6d.slurm.log
#SBATCH --output=fc_output6d.slurm.log
##SBATCH --cpus-per-task=5
##SBATCH --mem-per-cpu=8Gb
##SBATCH --ntasks=1 
##SBATCH --array=1
#SBATCH --partition=west


source activate cantera_env
python flame_speed_calc_6d.py
""".replace('label1',label1).replace('label2',label2)

    return bash_script




In [11]:
species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}


#list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
list_of_blends = ['CH2FCHF2_CH3F']

for file in list_of_blends: 
    
    try: 

        match = re.match('([A-Z0-9]+)_([A-Z0-9]+)', file) 
        label_1, label_2 = match.group(1), match.group(2)
        labels_we_hate={'CH2FCHF2':'C2H3F3','CH2FCH2F':'C2H4F2'}
        lower_1,upper_1,er_1 = species[label_1]
        lower_2,upper_2,er_2 = species[label_2]
        for i in labels_we_hate.keys():
            if label_1==i:
                label_1 = labels_we_hate[i]
            if label_2==i: 
                label_2 = labels_we_hate[i]
        print(label_1,label_2)
                
                
        label_3 = 'O2'
        print(lower_1,upper_1,lower_2,upper_2)

        #only graph overlapping region 
        final_lower = max(lower_1,lower_2)
        final_upper = min(upper_1,upper_2)
        print(final_lower, final_upper)

        #get species names 
        species_to_look_for = [i for i in species.keys() if not re.match('CH2FC[A-Z0-9]+',i)]
        species_to_look_for.append('C2H4F2')
        species_to_look_for.append('C2H3F3')
        species_to_look_for.append('O2')


        To = 298
        Po = 1e5 # ct.one_atm
        #chem_file = f'../{file}/cantera/chem.cti'
        chem_file = f'../{file}/chemkin/chem_final.cti'

        gas = ct.Solution(chem_file)
        print('made gas solution')
        species_names_we_want = {}
        for i in species_to_look_for:
            for name in gas.species_names:
                pattern = f"{i}\([0-9]+\)"
                m = re.match(pattern, name)
                if m:
                    species_names_we_want[i]=name
                    print(i, name)
                    break
            else:
                print(f"didnt find {i}")
        print(file, species_names_we_want)

        #now make the damn thing
        d = os.path.join(directory,file,'flame_speed_calc.py')
        if os.path.exists(d):
            continue
        flame_speed_calc += make_vol_frac_list(final_lower,final_upper)
        flame_speed_calc +=make_for_loop(species_names_we_want[label_1],species_names_we_want[label_2],species_names_we_want[label_3])
        bash_script = make_bash_script(label_1,label_2)
        d = os.path.join(directory,file)
        os.makedirs(d,exist_ok=True)
        print('here?')
        with open(os.path.join(d,'flame_speed_calc_6d.py'),'w') as f: 
            for l in flame_speed_calc: 
                f.write(l)
        with open(os.path.join(d,'flame_speed_run_6d.sh'),'w') as f: 
            for l in bash_script: 
                f.write(l)
        if os.path.exists(d): 
            continue 
        end_dir = 'data'
        data = os.path.join(d,end_dir,'')
        os.makedirs(data)  

    except:
        pass
    





C2H3F3 CH3F
0.075 0.15 0.08 0.225
0.08 0.15
made gas solution
CH2F2 CH2F2(41)
CH3F CH3F(2)
didnt find C2H5F
didnt find CH3CF3
didnt find CH3CHF2
didnt find C2H4F2
C2H3F3 C2H3F3(1)
O2 O2(3)
CH2FCHF2_CH3F {'CH2F2': 'CH2F2(41)', 'CH3F': 'CH3F(2)', 'C2H3F3': 'C2H3F3(1)', 'O2': 'O2(3)'}
here?
